## 처음 켯을때 이거 돌려주시고 하고나서 요코드 전부 선택하고 ctrl + / 해주세요. 돌릴때 앞에 샾이 있으면 코드가 안돌아가니 ctrl + / 해주시고 돌아갔으면 다시 ctrl + / 해서 # 있는상태로 돌려주세요

In [ ]:
# !pip install playwright
# !playwright install
# !pip install xlsxwriter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.5/35.5 MB 43.5 MB/s eta 0:00:00
155.8 Mb [] 0% 0.0s155.8 Mb [] 0% 34.1s155.8 Mb [] 0% 17.6s155.8 Mb [] 0% 11.2s155.8 Mb [] 0% 6.9s155.8 Mb [] 1% 5.7s155.8 Mb [] 1% 5.2s155.8 Mb [] 2% 4.4s155.8 Mb [] 3% 4.1s155.8 Mb [] 3% 3.8s155.8 Mb [] 4% 4.2s155.8 Mb [] 4% 3.7s155.8 Mb [] 5% 3.5s155.8 Mb [] 6% 3.4s155.8 Mb [] 7% 3.2s155.8 Mb [] 7% 3.1s155.8 Mb [] 8% 2.9s155.8 Mb [] 8% 3.0s155.8 Mb [] 9% 3.1s155.8 Mb [] 9% 3.0s155.8 Mb [] 10% 2.9s155.8 Mb [] 10% 3.1s155.8 Mb [] 11% 2.9s155.8 Mb [] 12% 2.8s155.8 Mb [] 13% 2.8s155.8 Mb [] 14% 2.7s155.8 Mb [] 15% 2.7s155.8 Mb [] 16% 2.7s155.8 Mb [] 17% 2.7s155.8 Mb [] 18% 2.6s155.8 Mb [] 19% 2.6s155.8 Mb [] 20% 2.5s155.8 Mb [] 21% 2.5s155.8 Mb [] 22% 2.4s155.8 Mb [] 23% 2.3s155.8 Mb [] 24% 2.3s155.8 Mb [] 26% 2.2s155.8 Mb [] 27% 2.2s155.8 Mb [] 28% 2.1s155.8 Mb [] 28% 2.2s155.8 Mb [] 28% 2.3s155.8 Mb [] 29% 2.3s155.8 Mb [] 30% 2.3s155.8 Mb [] 31% 2.2s155.8 Mb [] 32% 2.1s155.8 Mb [] 33% 2.1s155.8 Mb [] 34% 

#Imports and package installation

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import json
from datetime import datetime
from playwright.sync_api import sync_playwright
from google.colab import files
from openpyxl import Workbook
from openpyxl.styles import Font
import xlsxwriter
import numpy as np

# 여기다가 투어코드 쓰세요

In [ ]:
# Get input from the user
code = 'PAP221231107KEY' #@param {type:"string"}
location = '\uC2DC\uB4DC\uB2C8,\uACE8\uB4DC\uCF54\uC2A4\uD2B8' #@param {type:"string"}
# Split the input on commas and strip any leading/trailing whitespace
location = [item.strip() for item in location.split(',')]



In [ ]:
#Extract data from the hanatour API
#modify so that it's
# url = url
# payload = payload

def get_data(url,payload):
    url = url
    payload = payload

    querystring = {"_siteId":"hanatour"}
    payload = payload
    #Get these header details for your device from INSOMNIA
    headers = {
        "cookie": "",
        "Content-Type": "application/json",
        "Accept": "application/json, text/plain, */*",
        "Accept-Language": "en-GB,en-US;q=0.9,en;q=0.8",
        "Connection": "keep-alive",
        "Cookie": "",
        "Origin": "https://www.hanatour.com",
        "Referer": "https://www.hanatour.com/",
        "Sec-Fetch-Dest": "empty",
        "Sec-Fetch-Mode": "cors",
        "Sec-Fetch-Site": "same-site",
        "User-Agent": "",
        "prgmId": "",
        "sec-ch-ua": ""}
    response = requests.request("POST", url, json=payload, headers=headers, params=querystring)
    return response


# Other functions

In [ ]:
# @title arrival_flight_detail
#get arrival flight detail
def get_arrival_flight(code,departDate):
  url_flight_id = "https://api.hanatour.com/svc/mciPkg/getPkgProdInfo"
  payload_id = {"pkgCd":code,"inpPathCd":"DCP","smplYn":"N","coopYn":"N","resAcceptPtn":{},"partnerYn":"N"}
  response_id_data = get_data(url_flight_id,payload_id)
  response_id = json.loads(response_id_data.text)
  invIds = response_id["airInvInfo"]["invId"]

  url = "https://api.hanatour.com/svc/mciAir/schRtFlgt?_siteId=hanatour"
  payload = {"depDt":departDate,"invId":invIds}
  response = get_data(url,payload)
  response = json.loads(response.text)
  #get time and date
  date = response["rtFlgtLst"][0]["arrDate"]
  # time = response["rtFlgtLst"][0]["arrvTime"]
  # arrival_date = datetime.strptime(date, "%Y%m%d%")
  # arrival_time = datetime.strptime(time, "H%M")
  # time_12hr = arrival_time.strftime("%I:%M %p")
  # #get flight detail
  # flight_code = response["rtFlgtLst"][0]["mktAlCode"]+response["rtFlgtLst"][0]["mktAlNo"]

  # flight_detail_time = arrival_time + " " + flight_code

  return date




In [ ]:
# @title load_table

#make it into dataframe
def load_to_table(response,code):
  data = json.loads(response.text)

  #data["schdInfoList"][0]["schdMainInfoList"]
  table = pd.DataFrame(columns = ["Date","location","Activity"])

  start_date = datetime.strptime(data["schdInfoList"][0]["strtDt"],"%Y%m%d").date()
  location = data["pkgAirSeqList"][0]["depAptCityNm"]
  start_activity = data["pkgAirSeqList"][0]["airlCd"]+data["pkgAirSeqList"][0]["flgtNm"]+" 도착시간: " +data["pkgAirSeqList"][0]["arrHm"] +" " + data["pkgAirSeqList"][0]["arrAptNm"]
  #table = table.append([start_date, location,start_activity])
  for day in data["schdInfoList"]:
      date = datetime.strptime(day["strtDt"],"%Y%m%d").date()
      for event in day["schdMainInfoList"]:
          if event["arriveCityNm"] is not None:
              location = event["arriveCityNm"]
          if event["schdCatgNm"] == "선택관광":
              option_tour = event["cardNm"]
              price = event["cardCntntPc"]
              price_parse = BeautifulSoup(price,"html.parser")
              # Find all div elements with the class 'txt_cont'
              elements = price_parse.find_all('div', class_='txt_cont')
              # Loop through the elements and print the text if it contains '요금'
              option_price = ""
              for element in elements:
                  if '요금' in element.text:
                      option_price = element.text.strip()
              option_text = "옵션: "+option_tour + " " + option_price
              table.loc[len(table.index)] = [date,location,option_text]
          elif event["cmsInfoList"] is not None:
              for event_name in event["cmsInfoList"]:
                  table.loc[len(table.index)] = [date,location,event_name["cmsCntntNm"]]
          elif event["trfcMensNm"] == "항공":
              table.loc[len(table.index)] = [date,"항공",event["memoCont"]]
          elif event["trfcMensNm"] == "버스":
              bus_text = event["depCityNm"] + "-" + event["arriveCityNm"] +" 이동"
              table.loc[len(table.index)] = [date,location,bus_text]
          elif event["schdCatgNm"] == "식사":
              meal = event["dtlMealDvNm"] + " " + ('' if event["mealCont"] is None else event["mealCont"]) + ('' if event["mealTypeNm"] is None else event["mealTypeNm"])
              table.loc[len(table.index)] = [date,location,meal]
          elif event["memoTitlNm"] is not None:
              table.loc[len(table.index)] = [date,location,event["memoTitlNm"]]
      if day["htlInfoList"] is not None:
          hotel_detail = day["htlInfoList"][0]["htlAplcLangNm"]
          table.loc[len(table.index)] = [date,location,hotel_detail]

  depart_date = data["schdInfoList"][0]["strtDt"]
  aus_flt_arrival_detail = get_arrival_flight(code,depart_date)
  table["Date"][0] = datetime.strptime(aus_flt_arrival_detail,"%Y%m%d").date()
  table["location"][0] = "항공" #data["pkgAirSeqList"][0]["depAptCityNm"]
  arrival_time = datetime.strptime(data["pkgAirSeqList"][0]["arrHm"],"%H%M").strftime("%I:%M %p")
  table["Activity"][0] = data["pkgAirSeqList"][0]["airlCd"]+data["pkgAirSeqList"][0]["flgtNm"]+" 도착시간: " + arrival_time  +" " + data["pkgAirSeqList"][0]["arrAptNm"]

  end_date= datetime.strptime(data["schdInfoList"][-1]["strtDt"],"%Y%m%d").date()
  end_location = data["pkgAirSeqList"][-1]["depAptCityNm"]
  end_arrival_time = datetime.strptime(data["pkgAirSeqList"][-1]["depHm"],"%H%M").strftime("%I:%M %p")
  end_acitivity = data["pkgAirSeqList"][-1]["airlCd"]+data["pkgAirSeqList"][-1]["flgtNm"]+" 출발시간: " + end_arrival_time  +" " + data["pkgAirSeqList"][-1]["depAptNm"]
  table.loc[len(table.index)]=[end_date,end_location,end_acitivity]
  table.insert(1,"Time",None)
  table.insert(4,"Confirmation",None)

  return(table)

In [ ]:
# @title aggregate_region
#aggregate regions
def aggregate_region():
  region_aggregation = \
   {"뉴캐슬":"시드니",
    "포트스티븐스":"시드니",
    "울릉공":"시드니",
    "키아마": "시드니",
    "헌터벨리":"시드니",
    "저비스베이":"시드니",
    "커럼빈": "시드니",
    "시드니": "시드니",
    "골드코스트":"골드코스트",
    "브리즈번":"골드코스트",
    "멜버른":"멜버른"
    }
  return region_aggregation

In [ ]:
# @title separate_by_region
def new_separate_by_region(table,region):
  map_dict = aggregate_region()
    # Map the 'Location' column to the dictionary values
  table['Mapped_Location'] = table['location'].map(map_dict)

  # Filter rows where 'Mapped_Location' is 'region'
  filtered_df = table[(table['Mapped_Location'] == region) | (table['location']=="항공")]
  filtered_df.drop('Mapped_Location',axis = 1, inplace=True)
  return filtered_df



In [ ]:
# @title get_code_name
#get Code, code names, tc,tip amount,invId (They are all strings not numbers)
def get_code_names(code):
  response = get_data("https://api.hanatour.com/svc/mciPkg/getPkgProdInfo",{"pkgCd":code,"inpPathCd":"DCP","smplYn":"N","coopYn":"N","resAcceptPtn":{},"partnerYn":"N"})
  response = json.loads(response.text)
  code = response["saleProdCd"]
  code_name = response["saleProdNm"]
  tc = response["tcInfo"]["tcRmkCont"]
  tip = response["trvlExpnNoneInclList"][0]["trvlExpnDesc"]
  return [code, code_name, tc, tip]


In [ ]:
# @title get_num_people
#get number of people
def get_num_people(code):
  response = get_data("https://api.hanatour.com/svc/mciPkg/getPkgPromCdInfo?_siteId=hanatour",{"pkgCd":code,"qryDvCd":"1"})
  response = json.loads(response.text)
  return response["promCdInfo"][0]["pecnClpsnCnt"]

In [ ]:
# @title create_excel_file
#helper function
def highlight_rows(row, condition):
    if condition[row.name]:
        return [''] * len(row)
    else:
        return ['background-color: yellow'] * len(row)

# Make excel file and modify them
def create_excel_file(table,region,code):

  #keep the dates and location for first entry
  table.loc[table.duplicated(subset=['Date'], keep='first'), ['Date','location']]= None

  #Write into excel
  file_name = f"{code}_{region}.xlsx"
  writer = pd.ExcelWriter(file_name,engine = "xlsxwriter")
  table.to_excel(writer, sheet_name='Sheet1', index = False,startrow = 6)
  workbook  = writer.book
  worksheet = writer.sheets['Sheet1']

  #increase font size
    # Define a format for the font size
  font_format = workbook.add_format({'font_size': 14,})  # Change the number to your preferred font size

    # Define a format for the border
  border_format = workbook.add_format({'font_size': 14,'border':1})

    # Define a date format
  date_format = workbook.add_format({'num_format': 'yyyy-mm-dd','border':1,'font_size': 14})

  #auto fit column sizes and apply border
  for column in table:
    column_length = max(table[column].astype(str).map(len).max(), len(column))
    col_idx = table.columns.get_loc(column)
    worksheet.set_column(col_idx, col_idx, column_length,font_format)
        # Apply border format to cells in this column
    for row in range(7, len(table) + 7):  # Adjust the range according to your DataFrame's position
      worksheet.write(row, col_idx, table.iloc[row - 7, col_idx], border_format)

  # Apply the date format to the 'Date' column cells
  date_col_idx = table.columns.get_loc('Date')  # Get the index of 'Date' column
  for row in range(len(table)):  # Adjust the range according to your DataFrame's position
      worksheet.write(row + 7, date_col_idx, table.iloc[row, date_col_idx], date_format)

  #Write details above table
  bold_font = workbook.add_format({'bold':True,'font_size': 14})
  code, code_name, tc, tip = get_code_names(code)
  num_people = get_num_people(code)
  worksheet.write('A1', '그룹코드',bold_font)
  worksheet.write('B1', code,bold_font)
  worksheet.write('A2', code_name,bold_font)
  worksheet.write('A3', "가이드",bold_font)
  worksheet.write('A4', "총인원",bold_font)
  worksheet.write('B4', num_people)
  worksheet.write('A5', "인솔자",bold_font)
  worksheet.write('B5', tc)

  #Write details below the table
  num_rows = table.shape[0]

  worksheet.write(num_rows + 6 + 1, 0, "**비고사항**",bold_font)
  worksheet.write(num_rows + 6 + 2, 0, tip + " 시드니에서 걷습니다.")
  writer.close()
  files.download(file_name)








# main.py

In [ ]:
# @title main.py


url = "https://api.hanatour.com/svc/mciPkg/getPkgProdItnrInfo"
#url_tip_tc = "https://api.hanatour.com/svc/mciPkg/getPkgProdInfo"
data = get_data(url, {"pkgCd": code})

table = load_to_table(data,code)
#print(new_separate_by_region(table,"시드니"))
for region in location:
  create_excel_file(new_separate_by_region(table,region),region,code)

<ipython-input-8-7b9b59cb0cf1>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.drop('Mapped_Location',axis = 1, inplace=True)
<ipython-input-11-174ddf2f79c1>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table.loc[table.duplicated(subset=['Date'], keep='first'), ['Date','location']]= None


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<ipython-input-8-7b9b59cb0cf1>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.drop('Mapped_Location',axis = 1, inplace=True)
<ipython-input-11-174ddf2f79c1>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table.loc[table.duplicated(subset=['Date'], keep='first'), ['Date','location']]= None


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>